In [6]:
from dotenv import load_dotenv
import inspect
import os
import pandas as pd

from utils import init_flipside, execute_query

from queries import (
    bridge,
    dex,
    farcaster,
    lending,
    marketplace,
    nft,
    rwa,
    staking,
    community_curation,
    betting_prediction,
    meme,
    identity,
    donation,
    marketplace_zora,
    nft_zora,
    airdrop,
    payments
)

In [7]:
load_dotenv()
FLIPSIDE_API_KEY = os.getenv('FLIPSIDE_API_KEY')
flipside = init_flipside(FLIPSIDE_API_KEY)
apps = pd.read_csv('data/applications_reviewed.csv', index_col=0)

In [8]:
METRIC_MODULES = {
    'Art NFTs': {
        "module": nft,
        "params": ["address"]
    },
    'Other Media NFTs': {
        "module": nft,
        "params": ["address"]
    },
    'Bridges': {
        "module": bridge,
        "params": ["chain", "address"]
    },
    'DEX': {
        "module": dex,
        "params": ["chain", "address"]
    },
    'Channels': {
        "module": farcaster,
        "params": ["warpcast_url"]
    },
    'Frames': {
        "module": None, # needs to be updated
        "params": ["warpcast_url"]
    },
    'Lending': {
        "module": lending,
        "params": ["chain", "address"]
    },
    'Art Marketplace': {
        "module": marketplace,
        "params": ["chain", "address"]
    },
    'Other Media Marketplace': {
        "module": marketplace,
        "params": ["chain", "address"]
    },
    'Real World Assets': {
        "module": rwa,
        "params": ["chain", "address"]
    },
    'Staking': {
        "module": staking,
        "params": ["chain", "address"]
    },
    'Community & Curation': {
        "module": community_curation,
        "params": ["chain", "address"]
    },
    'Gaming': {
        "module": community_curation,
        "params": ["chain", "address"]
    },
    'Airdrop': {
        "module": airdrop,
        "params": ["chain", "address"]
    },
    'Meme Community': {
        "module": meme,
        "params": ["chain", "address"]
    },
    'Payments': {
        "module": payments,
        "params": ["chain", "address"]
    },    
    'Donations': {
        "module": donation,
        "params": ["chain", "address"]
    },
    'Identity': {
        "module": identity,
        "params": ["chain", "address"]
    },
    'Betting & Prediction Markets': {
        "module": betting_prediction,
        "params": ["chain", "address"]
    }
}

DUNE_METRIC_MODULES = {

        'Art NFTs': {
        "module": nft_zora,
        "params": ["address"]
    },
        'Other Media NFTs': {
        "module": nft_zora,
        "params": ["address"]
    },
    'Art Marketplace': {
        "module": marketplace_zora,
        "params": ["address"]
    },
    'Other Media Marketplace': {
        "module": marketplace_zora,
        "params": ["address"]
    }
    
}

In [13]:
def query_project_metrics(project_data):    
    uid = project_data['id']
    category = project_data['category']
    module_name = METRIC_MODULES.get(category).get('module')
    if not module_name:
        return
    
    metric_modules = inspect.getmembers(module_name, inspect.isfunction)
    expected_params = METRIC_MODULES.get(category).get('params', [])
    params = [project_data.get(p) for p in expected_params]
    if len(params) != len(expected_params):
        print("Unable to find params:", expected_params)
        return
    
    metrics = []
    project_info = {'id': uid, 'category': category}
    project_info.update(dict(zip(expected_params, params)))

    for (query_name, query_fn) in metric_modules:
        sql = query_fn(*params)
        value = execute_query(flipside, sql, project_info)
        metric_info = project_info.copy()
        metric_info.update({'query_name': query_name, 'value': value})
        metrics.append(metric_info)
    return metrics


def multichain_query_project_metrics( project_data):
    
    #metric is cumulative for multiple chain apps or creators
    metrics_cum = {}
    if project_data['chain'] == 'OP Mainnet':
        project_data['chain'] = 'optimism'
        metrics_cum = query_project_metrics( project_data)
        
    elif project_data['chain'] == 'All Superchain':
        
        for chain in ['optimism', 'base'] : #include zora after getting it cleared
            project_data['chain'] = chain
            metrics = query_project_metrics( project_data)
            
            for metric in metrics:
                if metric not in metrics_cum:
                    metrics_cum[metric] = metrics[metric]
                else:
                    metrics_cum[metric] += metrics[metric]
        
    return(metrics_cum)

In [9]:
apps.iloc[0]

id                        0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406...
recipient                        0xE4EE538019673501F4B75de5aF5CC073Ec0A1487
time                                                             1724860607
name                                                                     0x
status                                                             approved
profile_name                                                     ewokafloka
profile_url                                 https://warpcast.com/ewokafloka
metadata_name                                                            0x
metadata_website                                            https://0x.org/
project_type                                                            App
category                                                                DEX
address_type                                                       contract
address                          0xdef1c0ded9bec7f1a1670819833240f027b25eff
chain_id    

In [6]:
results_dex = query_project_metrics(apps.iloc[0])
results_dex

2024-09-18 19:54:26,818 - INFO - Query Attempt: id: 0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab, category: DEX, chain: Base, address: 0xdef1c0ded9bec7f1a1670819833240f027b25eff, Status: success, Execution Time: 70.60 seconds, Value: 1646413
2024-09-18 19:54:34,433 - INFO - Query Attempt: id: 0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab, category: DEX, chain: Base, address: 0xdef1c0ded9bec7f1a1670819833240f027b25eff, Status: success, Execution Time: 7.61 seconds, Value: 0
2024-09-18 19:55:34,038 - INFO - Query Attempt: id: 0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab, category: DEX, chain: Base, address: 0xdef1c0ded9bec7f1a1670819833240f027b25eff, Status: success, Execution Time: 59.60 seconds, Value: 27289
2024-09-18 19:55:51,938 - INFO - Query Attempt: id: 0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab, category: DEX, chain: Base, address: 0xdef1c0ded9bec7f1a1670819833240f027b25eff, Status: success,

[{'id': '0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab',
  'category': 'DEX',
  'chain': 'Base',
  'address': '0xdef1c0ded9bec7f1a1670819833240f027b25eff',
  'query_name': 'dex_ntrxns',
  'value': 1646413},
 {'id': '0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab',
  'category': 'DEX',
  'chain': 'Base',
  'address': '0xdef1c0ded9bec7f1a1670819833240f027b25eff',
  'query_name': 'dex_num_tokens',
  'value': 0},
 {'id': '0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab',
  'category': 'DEX',
  'chain': 'Base',
  'address': '0xdef1c0ded9bec7f1a1670819833240f027b25eff',
  'query_name': 'dex_nusers',
  'value': 27289},
 {'id': '0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab',
  'category': 'DEX',
  'chain': 'Base',
  'address': '0xdef1c0ded9bec7f1a1670819833240f027b25eff',
  'query_name': 'dex_sumtrxns',
  'value': 22188.398141304}]

In [6]:
apps.iloc[709]

id                        0x63c659c656d2fd865c7c4a19c74be68697c915f876a4...
recipient                        0xEAF56cF68C38281252E49b460c5a66925991475B
time                                                             1724764677
name                                                                 L2Pass
status                                                              pending
profile_name                                                          khuan
profile_url                                      https://warpcast.com/khuan
metadata_name                                                        L2Pass
metadata_website                                        https://l2pass.com/
project_type                                                            App
category                                                            Bridges
address_type                                                       contract
address                          0x0000049f63ef0d60abe49fdd8bebfa5a68822222
chain_id    

In [7]:
# bridges testing

results_bridges = query_project_metrics(apps.iloc[709])
results_bridges

2024-09-18 20:34:24,216 - INFO - Query Attempt: id: 0x63c659c656d2fd865c7c4a19c74be68697c915f876a45bc893af5931239040df, category: Bridges, chain: Base, address: 0x0000049f63ef0d60abe49fdd8bebfa5a68822222, Status: success, Execution Time: 11.11 seconds, Value: None
2024-09-18 20:34:28,339 - INFO - Query Attempt: id: 0x63c659c656d2fd865c7c4a19c74be68697c915f876a45bc893af5931239040df, category: Bridges, chain: Base, address: 0x0000049f63ef0d60abe49fdd8bebfa5a68822222, Status: success, Execution Time: 4.12 seconds, Value: 0
2024-09-18 20:34:32,132 - INFO - Query Attempt: id: 0x63c659c656d2fd865c7c4a19c74be68697c915f876a45bc893af5931239040df, category: Bridges, chain: Base, address: 0x0000049f63ef0d60abe49fdd8bebfa5a68822222, Status: success, Execution Time: 3.79 seconds, Value: 0
2024-09-18 20:34:40,897 - INFO - Query Attempt: id: 0x63c659c656d2fd865c7c4a19c74be68697c915f876a45bc893af5931239040df, category: Bridges, chain: Base, address: 0x0000049f63ef0d60abe49fdd8bebfa5a68822222, Status: 

[{'id': '0x63c659c656d2fd865c7c4a19c74be68697c915f876a45bc893af5931239040df',
  'category': 'Bridges',
  'chain': 'Base',
  'address': '0x0000049f63ef0d60abe49fdd8bebfa5a68822222',
  'query_name': 'bridge_medianfee',
  'value': None},
 {'id': '0x63c659c656d2fd865c7c4a19c74be68697c915f876a45bc893af5931239040df',
  'category': 'Bridges',
  'chain': 'Base',
  'address': '0x0000049f63ef0d60abe49fdd8bebfa5a68822222',
  'query_name': 'bridge_nbtrxns',
  'value': 0},
 {'id': '0x63c659c656d2fd865c7c4a19c74be68697c915f876a45bc893af5931239040df',
  'category': 'Bridges',
  'chain': 'Base',
  'address': '0x0000049f63ef0d60abe49fdd8bebfa5a68822222',
  'query_name': 'bridge_nchains',
  'value': 0},
 {'id': '0x63c659c656d2fd865c7c4a19c74be68697c915f876a45bc893af5931239040df',
  'category': 'Bridges',
  'chain': 'Base',
  'address': '0x0000049f63ef0d60abe49fdd8bebfa5a68822222',
  'query_name': 'bridge_nusers',
  'value': 0},
 {'id': '0x63c659c656d2fd865c7c4a19c74be68697c915f876a45bc893af5931239040df'

In [9]:
apps.iloc[65]

id                        0x855b930c39eb78fe9ab6929f03dcab557ba3fed5600c...
recipient                        0x0000000000000000000000000000000000000000
time                                                             1724724419
name                                                        Arcadia Finance
status                                                              pending
profile_name                                                           kawz
profile_url                                       https://warpcast.com/kawz
metadata_name                                               Arcadia Finance
metadata_website                                    https://arcadia.finance
project_type                                                            App
category                                                            Lending
address_type                                                       contract
address                          0xda14fdd72345c4d2511357214c5b89a919768e59
chain_id    

In [11]:
#lending testing

results_lending = query_project_metrics(apps.iloc[65])
results_lending

2024-09-18 20:42:54,345 - INFO - Query Attempt: id: 0x855b930c39eb78fe9ab6929f03dcab557ba3fed5600cdc87e4bc9539fa790ed5, category: Lending, chain: Base, address: 0xda14fdd72345c4d2511357214c5b89a919768e59, Status: success, Execution Time: 3.95 seconds, Value: 0
2024-09-18 20:42:58,130 - INFO - Query Attempt: id: 0x855b930c39eb78fe9ab6929f03dcab557ba3fed5600cdc87e4bc9539fa790ed5, category: Lending, chain: Base, address: 0xda14fdd72345c4d2511357214c5b89a919768e59, Status: success, Execution Time: 3.78 seconds, Value: 0
2024-09-18 20:43:01,914 - INFO - Query Attempt: id: 0x855b930c39eb78fe9ab6929f03dcab557ba3fed5600cdc87e4bc9539fa790ed5, category: Lending, chain: Base, address: 0xda14fdd72345c4d2511357214c5b89a919768e59, Status: success, Execution Time: 3.78 seconds, Value: 0
2024-09-18 20:43:08,660 - INFO - Query Attempt: id: 0x855b930c39eb78fe9ab6929f03dcab557ba3fed5600cdc87e4bc9539fa790ed5, category: Lending, chain: Base, address: 0xda14fdd72345c4d2511357214c5b89a919768e59, Status: succ

[{'id': '0x855b930c39eb78fe9ab6929f03dcab557ba3fed5600cdc87e4bc9539fa790ed5',
  'category': 'Lending',
  'chain': 'Base',
  'address': '0xda14fdd72345c4d2511357214c5b89a919768e59',
  'query_name': 'lending_nborrowers',
  'value': 0},
 {'id': '0x855b930c39eb78fe9ab6929f03dcab557ba3fed5600cdc87e4bc9539fa790ed5',
  'category': 'Lending',
  'chain': 'Base',
  'address': '0xda14fdd72345c4d2511357214c5b89a919768e59',
  'query_name': 'lending_nproviders',
  'value': 0},
 {'id': '0x855b930c39eb78fe9ab6929f03dcab557ba3fed5600cdc87e4bc9539fa790ed5',
  'category': 'Lending',
  'chain': 'Base',
  'address': '0xda14fdd72345c4d2511357214c5b89a919768e59',
  'query_name': 'lending_ntrxns',
  'value': 0},
 {'id': '0x855b930c39eb78fe9ab6929f03dcab557ba3fed5600cdc87e4bc9539fa790ed5',
  'category': 'Lending',
  'chain': 'Base',
  'address': '0xda14fdd72345c4d2511357214c5b89a919768e59',
  'query_name': 'lending_valtrxns',
  'value': None}]

In [14]:
apps.iloc[1325]

id                        0xdcf5097af0e4e4d37f5b916e91e0a565a388888ad57f...
recipient                        0x37A6156e4a6E8B60b2415AF040546cF5e91699bd
time                                                             1724765453
name                                                         Vesper Finance
status                                                             approved
profile_name                                                      greenjeff
profile_url                                  https://warpcast.com/greenjeff
metadata_name                                                Vesper Finance
metadata_website                                    https://vesper.finance/
project_type                                                            App
category                                                            Staking
address_type                                                       contract
address                          0x32934ad7b1121defc631080b58599a0eaab89878
chain_id    

In [27]:
#staking testing

results_staking = query_project_metrics(apps.iloc[1325])
results_staking

['OP Mainnet', '0x32934ad7b1121defc631080b58599a0eaab89878']


2024-09-18 21:40:02,969 - INFO - Query Attempt: id: 0xdcf5097af0e4e4d37f5b916e91e0a565a388888ad57fda7f6ae1d3d91f1577c2, category: Staking, chain: OP Mainnet, address: 0x32934ad7b1121defc631080b58599a0eaab89878, Status: success, Execution Time: 7.84 seconds, Value: 3
2024-09-18 21:40:06,795 - INFO - Query Attempt: id: 0xdcf5097af0e4e4d37f5b916e91e0a565a388888ad57fda7f6ae1d3d91f1577c2, category: Staking, chain: OP Mainnet, address: 0x32934ad7b1121defc631080b58599a0eaab89878, Status: success, Execution Time: 3.82 seconds, Value: 0
2024-09-18 21:40:10,766 - INFO - Query Attempt: id: 0xdcf5097af0e4e4d37f5b916e91e0a565a388888ad57fda7f6ae1d3d91f1577c2, category: Staking, chain: OP Mainnet, address: 0x32934ad7b1121defc631080b58599a0eaab89878, Status: success, Execution Time: 3.97 seconds, Value: 0


[{'id': '0xdcf5097af0e4e4d37f5b916e91e0a565a388888ad57fda7f6ae1d3d91f1577c2',
  'category': 'Staking',
  'chain': 'OP Mainnet',
  'address': '0x32934ad7b1121defc631080b58599a0eaab89878',
  'query_name': 'staking_ntrxns',
  'value': 3},
 {'id': '0xdcf5097af0e4e4d37f5b916e91e0a565a388888ad57fda7f6ae1d3d91f1577c2',
  'category': 'Staking',
  'chain': 'OP Mainnet',
  'address': '0x32934ad7b1121defc631080b58599a0eaab89878',
  'query_name': 'staking_nusers',
  'value': 0},
 {'id': '0xdcf5097af0e4e4d37f5b916e91e0a565a388888ad57fda7f6ae1d3d91f1577c2',
  'category': 'Staking',
  'chain': 'OP Mainnet',
  'address': '0x32934ad7b1121defc631080b58599a0eaab89878',
  'query_name': 'staking_sumtrxns',
  'value': 0}]

In [24]:
apps.iloc[1]

id                        0x5a587244ecf18246e881cda06bac1ece7c0995bb2faf...
recipient                        0x0000000000000000000000000000000000000000
time                                                             1724723573
name                                                           1001 raisons
status                                                             approved
profile_name                                                           kawz
profile_url                                       https://warpcast.com/kawz
metadata_name                                                  1001 raisons
metadata_website                       https://www.sound.xyz/medmc/releases
project_type                                                        Creator
category                                                           Art NFTs
address_type                                                  mintingWallet
address                          0x8f59aa0f586d6d941152b9075b344cdc42e9b024
chain_id    

In [35]:
#creator Art NFTS testing

results_nft = query_project_metrics(apps.iloc[1])
results_nft

2024-09-18 21:50:47,761 - INFO - Query Attempt: id: 0x5a587244ecf18246e881cda06bac1ece7c0995bb2faf303f967d450f3500afdc, category: Art NFTs, address: 0x8f59aa0f586d6d941152b9075b344cdc42e9b024, Status: success, Execution Time: 10.52 seconds, Value: 0
2024-09-18 21:50:51,753 - INFO - Query Attempt: id: 0x5a587244ecf18246e881cda06bac1ece7c0995bb2faf303f967d450f3500afdc, category: Art NFTs, address: 0x8f59aa0f586d6d941152b9075b344cdc42e9b024, Status: success, Execution Time: 3.99 seconds, Value: 146604


[{'id': '0x5a587244ecf18246e881cda06bac1ece7c0995bb2faf303f967d450f3500afdc',
  'category': 'Art NFTs',
  'address': '0x8f59aa0f586d6d941152b9075b344cdc42e9b024',
  'query_name': 'artnft_nam',
  'value': 0},
 {'id': '0x5a587244ecf18246e881cda06bac1ece7c0995bb2faf303f967d450f3500afdc',
  'category': 'Art NFTs',
  'address': '0x8f59aa0f586d6d941152b9075b344cdc42e9b024',
  'query_name': 'artnft_ndrops',
  'value': 146604}]

In [17]:
apps.iloc[1106]

id                        0x57debdaeba9e245c5665cfaf221b82b65a90656b9680...
recipient                        0x9f5E9A4934929D59231d905A8A0554807CC3dB08
time                                                             1724855457
name                                                          sarahdigilife
status                                                              pending
profile_name                                                  sarahdigilife
profile_url                              https://warpcast.com/sarahdigilife
metadata_name                                                 sarahdigilife
metadata_website                           https://linktr.ee/sarah_digilife
project_type                                                        Creator
category                                                    Art Marketplace
address_type                                                  mintingWallet
address                          0x9f5e9a4934929d59231d905a8a0554807cc3db08
chain_id    

In [31]:
#nft marketplace testing

results_nftMarketplace = query_project_metrics(apps.iloc[1106])
results_nftMarketplace

2024-10-10 06:28:10,643 - INFO - Query Attempt: id: 0x57debdaeba9e245c5665cfaf221b82b65a90656b9680cc2bc735e6d826ff54fa, category: Art Marketplace, chain: All Superchain, address: 0x9f5e9a4934929d59231d905a8a0554807cc3db08, Status: success, Execution Time: 6.40 seconds, Value: 0
2024-10-10 06:28:15,126 - INFO - Query Attempt: id: 0x57debdaeba9e245c5665cfaf221b82b65a90656b9680cc2bc735e6d826ff54fa, category: Art Marketplace, chain: All Superchain, address: 0x9f5e9a4934929d59231d905a8a0554807cc3db08, Status: success, Execution Time: 4.44 seconds, Value: 0
2024-10-10 06:28:19,567 - INFO - Query Attempt: id: 0x57debdaeba9e245c5665cfaf221b82b65a90656b9680cc2bc735e6d826ff54fa, category: Art Marketplace, chain: All Superchain, address: 0x9f5e9a4934929d59231d905a8a0554807cc3db08, Status: success, Execution Time: 4.44 seconds, Value: 0
2024-10-10 06:28:23,978 - INFO - Query Attempt: id: 0x57debdaeba9e245c5665cfaf221b82b65a90656b9680cc2bc735e6d826ff54fa, category: Art Marketplace, chain: All Super

[{'id': '0x57debdaeba9e245c5665cfaf221b82b65a90656b9680cc2bc735e6d826ff54fa',
  'category': 'Art Marketplace',
  'chain': 'All Superchain',
  'address': '0x9f5e9a4934929d59231d905a8a0554807cc3db08',
  'query_name': 'artmarketplace_nac',
  'value': 0},
 {'id': '0x57debdaeba9e245c5665cfaf221b82b65a90656b9680cc2bc735e6d826ff54fa',
  'category': 'Art Marketplace',
  'chain': 'All Superchain',
  'address': '0x9f5e9a4934929d59231d905a8a0554807cc3db08',
  'query_name': 'artmarketplace_nam',
  'value': 0},
 {'id': '0x57debdaeba9e245c5665cfaf221b82b65a90656b9680cc2bc735e6d826ff54fa',
  'category': 'Art Marketplace',
  'chain': 'All Superchain',
  'address': '0x9f5e9a4934929d59231d905a8a0554807cc3db08',
  'query_name': 'artmarketplace_ntrxns',
  'value': 0},
 {'id': '0x57debdaeba9e245c5665cfaf221b82b65a90656b9680cc2bc735e6d826ff54fa',
  'category': 'Art Marketplace',
  'chain': 'All Superchain',
  'address': '0x9f5e9a4934929d59231d905a8a0554807cc3db08',
  'query_name': 'artmarketplace_valtrxns',

In [35]:
apps.iloc[10]

id                        0x0d9957d5b0802c372624b35145aa72953e0ddcfacf28...
recipient                        0x0000000000000000000000000000000000000000
time                                                             1725288949
name                                                              777 Enjoy
status                                                             approved
profile_name                                                           kawz
profile_url                                       https://warpcast.com/kawz
metadata_name                                                     777 Enjoy
metadata_website          https://zora.co/collect/zora:0xee9dcb1227ca891...
project_type                                                        Creator
category                                               Community & Curation
address_type                                                  mintingWallet
address                          0x5c67c59c850afb2fb2aace4c3e03a222b992266c
chain_id    

In [7]:
#community_curation

results_community = query_project_metrics(apps.iloc[10])
results_community


    WITH user_transactions AS (
        SELECT 
            t.from_address,
            t.block_timestamp,
            COUNT(*) OVER (
                PARTITION BY t.from_address 
                ORDER BY t.block_timestamp 
                RANGE BETWEEN INTERVAL '60 DAY' PRECEDING AND CURRENT ROW
            ) AS tx_count_last_Y_days
        FROM base.core.fact_transactions t
        WHERE t.to_address = '0x5c67c59c850afb2fb2aace4c3e03a222b992266c'
        AND t.block_timestamp BETWEEN '2024-06-01' AND '2024-09-01'
        
    ),
    active_users AS (
        SELECT DISTINCT from_address as active_user
        FROM user_transactions
        WHERE tx_count_last_Y_days >= 5
    ),
    flattened_addresses AS (
    SELECT f.fid, flattened.VALUE AS verified_address
    FROM external.farcaster.dim_profile_with_addresses f,
    TABLE(FLATTEN(INPUT => SPLIT(f.VERIFIED_ADDRESSES, ','))) AS flattened
        )
        SELECT COUNT(DISTINCT AU.active_user)
        FROM active_users AU
        J

[]

In [46]:
apps.iloc[208]

id                        0x04f2233951090d71b8808b4ec44c5420cbb51ab4de3f...
recipient                        0x013437412c41716f72B54fE66d7c11858aFe4894
time                                                             1724769997
name                                                                BetBase
status                                                             approved
profile_name                                                    betbase-xyz
profile_url                                https://warpcast.com/betbase-xyz
metadata_name                                                       BetBase
metadata_website                                       https://betbase.xyz/
project_type                                                            App
category                                       Betting & Prediction Markets
address_type                                                       contract
address                          0x80f6bcedd3d4fa1035285affa30e38f464db3895
chain_id    

In [5]:
#betting and prediction markets

results_betting = query_project_metrics(apps.iloc[208])
results_betting

2024-10-10 07:52:07,789 - INFO - Query Attempt: id: 0x04f2233951090d71b8808b4ec44c5420cbb51ab4de3f6cd174b07fe65a8cfa57, category: Betting & Prediction Markets, chain: Base, address: 0x80f6bcedd3d4fa1035285affa30e38f464db3895, Status: success, Execution Time: 6.97 seconds, Value: 1979
2024-10-10 07:52:17,695 - INFO - Query Attempt: id: 0x04f2233951090d71b8808b4ec44c5420cbb51ab4de3f6cd174b07fe65a8cfa57, category: Betting & Prediction Markets, chain: Base, address: 0x80f6bcedd3d4fa1035285affa30e38f464db3895, Status: success, Execution Time: 9.90 seconds, Value: 60
2024-10-10 07:52:24,551 - INFO - Query Attempt: id: 0x04f2233951090d71b8808b4ec44c5420cbb51ab4de3f6cd174b07fe65a8cfa57, category: Betting & Prediction Markets, chain: Base, address: 0x80f6bcedd3d4fa1035285affa30e38f464db3895, Status: success, Execution Time: 6.86 seconds, Value: 0


[{'id': '0x04f2233951090d71b8808b4ec44c5420cbb51ab4de3f6cd174b07fe65a8cfa57',
  'category': 'Betting & Prediction Markets',
  'chain': 'Base',
  'address': '0x80f6bcedd3d4fa1035285affa30e38f464db3895',
  'query_name': 'bp_ntrxns',
  'value': 1979},
 {'id': '0x04f2233951090d71b8808b4ec44c5420cbb51ab4de3f6cd174b07fe65a8cfa57',
  'category': 'Betting & Prediction Markets',
  'chain': 'Base',
  'address': '0x80f6bcedd3d4fa1035285affa30e38f464db3895',
  'query_name': 'bp_nusers',
  'value': 60},
 {'id': '0x04f2233951090d71b8808b4ec44c5420cbb51ab4de3f6cd174b07fe65a8cfa57',
  'category': 'Betting & Prediction Markets',
  'chain': 'Base',
  'address': '0x80f6bcedd3d4fa1035285affa30e38f464db3895',
  'query_name': 'bp_sumtrxns',
  'value': 0}]

In [6]:
apps.iloc[95]

id                        0x0f236979363275877d46ddccc57514af5a8cc8a9ee48...
recipient                        0xAa66fC4433DFD254F05A720A8c8a2f1F2fbDCB58
time                                                             1724725467
name                                                            baby Dragon
status                                                              pending
profile_name                                                     eldarfaaiz
profile_url                                 https://warpcast.com/eldarfaaiz
metadata_name                                                   baby Dragon
metadata_website                                    https://dragonbase.baby
project_type                                                            App
category                                                     Meme Community
address_type                                                       contract
address                          0xa5f8daa537afeca842cdd39758744fd3155a26cb
chain_id    

In [5]:
apps[apps['category']=='Meme Community']

,id,recipient,time,name,status,profile_name,profile_url,metadata_name,metadata_website,project_type,category,address_type,address,chain_id,chain,auto_validation,passed_auto_validation
95,0x0f236979363275877d46ddccc57514af5a8cc8a9ee48...,0xAa66fC4433DFD254F05A720A8c8a2f1F2fbDCB58,1724725467,baby Dragon,pending,eldarfaaiz,https://warpcast.com/eldarfaaiz,baby Dragon,https://dragonbase.baby,App,Meme Community,contract,0xa5f8daa537afeca842cdd39758744fd3155a26cb,8453.0,Base,OK,True
126,0xc6ac7c5193158b049cfb60bdd7c2b8a0495b690631ab...,0x6E23873AB8dD6Fb4823dF8A92D43643Fff537684,1725561237,BasedChad,pending,basedchad69,https://warpcast.com/basedchad69,BasedChad,https://basedchad.tech,Other Application,Meme Community,NaN,NaN,NaN,NaN,Reject - Missing/invalid address,False
127,0x20bcd145876dbe5b71ea89ac59867d4d077c3f37f9d8...,0x6E23873AB8dD6Fb4823dF8A92D43643Fff537684,1725561413,BasedChad,pending,basedchad69,https://warpcast.com/basedchad69,BasedChad,https://basedchad.tech,App,Meme Community,contract,0x17931cfc3217261ce0fa21bb066633c463ed8634,8453.0,Base,OK,True
141,0xa8ce02b54549cae0472cdfc3a003a87d4dac12d5537c...,0xD3228B3a7256919a31c18b935d752f24D6ED563f,1725113403,BASED MARIO,pending,ramiflix,https://warpcast.com/ramiflix,BASED MARIO,https://bario.buzz,Other Application,Meme Community,NaN,NaN,NaN,NaN,Reject - Missing/invalid address,False
149,0xd06585eb82305bf819c095afbe57098c43cdd4dcdc01...,0x75C83356987c8d813829D9FBb5DE504b547750A6,1724767291,Based St◨◨dio,pending,goyabean.eth,https://warpcast.com/goyabean.eth,Based St◨◨dio,http://noggleit.wtf/,Other Application,Meme Community,NaN,NaN,NaN,NaN,Reject - Missing/invalid address,False
188,0x1eb0aa94adf2a68a0f0d572b20bdb06b6146702f678d...,0x60ba240114b825F676D11Cc178409Cc9c385171c,1725479225,Base-Yoruba,pending,onbloaded,https://warpcast.com/onbloaded,Base-Yoruba,HTTPS://baseyoruba.com,Other Application,Meme Community,NaN,NaN,NaN,NaN,Reject - Missing/invalid address,False
195,0x73e61d0cde9f5d88d430f7b6315418dd8c5419c161be...,0x823c6115651D915614EB8d9f899d6d1FA939936D,1724763819,$BDOOGS,pending,livins94,https://warpcast.com/livins94,$BDOOGS,https://wagmi.best/0xFCB65dC6758E2cFd8f1dCdfad...,App,Meme Community,contract,0xfcb65dc6758e2cfd8f1dcdfad185bd4648b0e9c5,8453.0,Base,OK,True
220,0xea53703efa4ccaea7814ef95b215169c09678249fe91...,0x0000000000000000000000000000000000000000,1725019111,BLOBz,approved,kawz,https://warpcast.com/kawz,BLOBz,https://www.blobz.xyz/,App,Meme Community,contract,0x128a1e35398c72a9a060f482963f594752beccef,10.0,OP Mainnet,OK,True
237,0xe3c4817b00bdf6104bb41a3294c83c93e484624f29a6...,0x16420FC422F1Ae747Cc6BbC7075C466efBA07b4d,1725971149,Bond,pending,konstiantyn,https://warpcast.com/konstiantyn,Bond,https://www.youtube.com/@kostyadobosh81,App,Meme Community,contract,0x35706dfbe28425dc831af3517b41b176e009cd0d,8453.0,Base,OK,True
404,0x0c65af94143bb893fc7fd972e257c6ccdae61bf45f36...,0x330875BED67074B0c261Ec147Ab535806E21EE2D,1725619631,doginme memecoin,approved,doginme,https://warpcast.com/doginme,doginme memecoin,https://dogin.meme,Other Application,Meme Community,contract,0x6921b130d297cc43754afba22e5eac0fbf8db75b,8453.0,Base,OK,True


In [5]:
#meme community

results_meme = query_project_metrics(apps.iloc[95])
results_meme

2024-10-10 08:02:11,299 - INFO - Query Attempt: id: 0x0f236979363275877d46ddccc57514af5a8cc8a9ee48b6c9d089e32aaa57adba, category: Meme Community, chain: Base, address: 0xa5f8daa537afeca842cdd39758744fd3155a26cb, Status: success, Execution Time: 6.74 seconds, Value: 0
2024-10-10 08:02:17,953 - INFO - Query Attempt: id: 0x0f236979363275877d46ddccc57514af5a8cc8a9ee48b6c9d089e32aaa57adba, category: Meme Community, chain: Base, address: 0xa5f8daa537afeca842cdd39758744fd3155a26cb, Status: success, Execution Time: 6.65 seconds, Value: 1
2024-10-10 08:02:22,394 - INFO - Query Attempt: id: 0x0f236979363275877d46ddccc57514af5a8cc8a9ee48b6c9d089e32aaa57adba, category: Meme Community, chain: Base, address: 0xa5f8daa537afeca842cdd39758744fd3155a26cb, Status: success, Execution Time: 4.44 seconds, Value: 0
2024-10-10 08:02:26,908 - INFO - Query Attempt: id: 0x0f236979363275877d46ddccc57514af5a8cc8a9ee48b6c9d089e32aaa57adba, category: Meme Community, chain: Base, address: 0xa5f8daa537afeca842cdd39758

[{'id': '0x0f236979363275877d46ddccc57514af5a8cc8a9ee48b6c9d089e32aaa57adba',
  'category': 'Meme Community',
  'chain': 'Base',
  'address': '0xa5f8daa537afeca842cdd39758744fd3155a26cb',
  'query_name': 'meme_fnusers',
  'value': 0},
 {'id': '0x0f236979363275877d46ddccc57514af5a8cc8a9ee48b6c9d089e32aaa57adba',
  'category': 'Meme Community',
  'chain': 'Base',
  'address': '0xa5f8daa537afeca842cdd39758744fd3155a26cb',
  'query_name': 'meme_ntrxns',
  'value': 1},
 {'id': '0x0f236979363275877d46ddccc57514af5a8cc8a9ee48b6c9d089e32aaa57adba',
  'category': 'Meme Community',
  'chain': 'Base',
  'address': '0xa5f8daa537afeca842cdd39758744fd3155a26cb',
  'query_name': 'meme_nusers',
  'value': 0},
 {'id': '0x0f236979363275877d46ddccc57514af5a8cc8a9ee48b6c9d089e32aaa57adba',
  'category': 'Meme Community',
  'chain': 'Base',
  'address': '0xa5f8daa537afeca842cdd39758744fd3155a26cb',
  'query_name': 'meme_sumtrxns',
  'value': 0}]

In [7]:
apps.iloc[383]

id                        0xef60c3ab6436942a10e9bb0197b2f1e31d7d98be3cd5...
recipient                        0x2462B983E444B317C999b85CE3BBa79840D36F76
time                                                             1724762171
name                                                     Degen Name Service
status                                                              pending
profile_name                                                wen-airdrop.eth
profile_url                            https://warpcast.com/wen-airdrop.eth
metadata_name                                            Degen Name Service
metadata_website                         https://freename.io/discover/degen
project_type                                                            App
category                                                           Identity
address_type                                                       contract
address                          0x465ea4967479a96d4490d575b5a6cc2b4a4bee65
chain_id    

In [6]:
#identity

results_identity = query_project_metrics(apps.iloc[383])
results_identity

2024-10-10 08:04:55,370 - INFO - Query Attempt: id: 0xef60c3ab6436942a10e9bb0197b2f1e31d7d98be3cd5e0a64f1091fa1857d069, category: Identity, chain: Base, address: 0x465ea4967479a96d4490d575b5a6cc2b4a4bee65, Status: success, Execution Time: 10.77 seconds, Value: 2


[{'id': '0xef60c3ab6436942a10e9bb0197b2f1e31d7d98be3cd5e0a64f1091fa1857d069',
  'category': 'Identity',
  'chain': 'Base',
  'address': '0x465ea4967479a96d4490d575b5a6cc2b4a4bee65',
  'query_name': 'identity_nusers',
  'value': 2}]

In [9]:
apps.iloc[8]

id                        0xc6ef0dcb93e384cec4e159c7c53e218455eddb16c00d...
recipient                        0xA89946F00052Bc385B3121b45420316a7cE20E60
time                                                             1725643215
name                                                                 5050IO
status                                                              pending
profile_name                                                    kingsimpa69
profile_url                                https://warpcast.com/kingsimpa69
metadata_name                                                        5050IO
metadata_website                                           https://50-50.io
project_type                                                            App
category                                                          Donations
address_type                                                       contract
address                          0x0ce9a74dc442b8b7dc4b3edd5833cad8ac52afb9
chain_id    

In [6]:
#donation

results_donation = query_project_metrics(apps.iloc[8])
results_donation

2024-10-10 08:07:50,899 - INFO - Query Attempt: id: 0xc6ef0dcb93e384cec4e159c7c53e218455eddb16c00d6e12cf525811a4debcf8, category: Donations, chain: Base, address: 0x0ce9a74dc442b8b7dc4b3edd5833cad8ac52afb9, Status: success, Execution Time: 9.93 seconds, Value: 10
2024-10-10 08:08:01,094 - INFO - Query Attempt: id: 0xc6ef0dcb93e384cec4e159c7c53e218455eddb16c00d6e12cf525811a4debcf8, category: Donations, chain: Base, address: 0x0ce9a74dc442b8b7dc4b3edd5833cad8ac52afb9, Status: success, Execution Time: 10.19 seconds, Value: 0
2024-10-10 08:08:10,648 - INFO - Query Attempt: id: 0xc6ef0dcb93e384cec4e159c7c53e218455eddb16c00d6e12cf525811a4debcf8, category: Donations, chain: Base, address: 0x0ce9a74dc442b8b7dc4b3edd5833cad8ac52afb9, Status: success, Execution Time: 9.55 seconds, Value: 0.005


[{'id': '0xc6ef0dcb93e384cec4e159c7c53e218455eddb16c00d6e12cf525811a4debcf8',
  'category': 'Donations',
  'chain': 'Base',
  'address': '0x0ce9a74dc442b8b7dc4b3edd5833cad8ac52afb9',
  'query_name': 'donation_ntrxns',
  'value': 10},
 {'id': '0xc6ef0dcb93e384cec4e159c7c53e218455eddb16c00d6e12cf525811a4debcf8',
  'category': 'Donations',
  'chain': 'Base',
  'address': '0x0ce9a74dc442b8b7dc4b3edd5833cad8ac52afb9',
  'query_name': 'donation_nusers',
  'value': 0},
 {'id': '0xc6ef0dcb93e384cec4e159c7c53e218455eddb16c00d6e12cf525811a4debcf8',
  'category': 'Donations',
  'chain': 'Base',
  'address': '0x0ce9a74dc442b8b7dc4b3edd5833cad8ac52afb9',
  'query_name': 'donation_sumtrxns',
  'value': 0.005}]

In [8]:
apps.iloc[35]

id                        0xc03104f0878b4aeb1dcbd5395076e2e8b846b6e113e4...
recipient                        0x6048ac7Bb25389994AE13EC9aD40cdDFdc33Dc0A
time                                                             1725297111
name                                                             Airdropers
status                                                              pending
profile_name                                                      blockduck
profile_url                                  https://warpcast.com/blockduck
metadata_name                                                    Airdropers
metadata_website                             https://warpcast.com/blockduck
project_type                                                        Creator
category                                                            Airdrop
address_type                                                  mintingWallet
address                          0x6048ac7bb25389994ae13ec9ad40cddfdc33dc0a
chain_id    

In [6]:
#airdrop

results_airdrop = query_project_metrics(apps.iloc[35])
results_airdrop

2024-10-10 08:18:18,481 - INFO - Query Attempt: id: 0xc03104f0878b4aeb1dcbd5395076e2e8b846b6e113e434322705c61c63be622e, category: Airdrop, chain: All Superchain, address: 0x6048ac7bb25389994ae13ec9ad40cddfdc33dc0a, Status: success, Execution Time: 9.91 seconds, Value: 1
2024-10-10 08:18:22,869 - INFO - Query Attempt: id: 0xc03104f0878b4aeb1dcbd5395076e2e8b846b6e113e434322705c61c63be622e, category: Airdrop, chain: All Superchain, address: 0x6048ac7bb25389994ae13ec9ad40cddfdc33dc0a, Status: success, Execution Time: 4.39 seconds, Value: 0.23208187


[{'id': '0xc03104f0878b4aeb1dcbd5395076e2e8b846b6e113e434322705c61c63be622e',
  'category': 'Airdrop',
  'chain': 'All Superchain',
  'address': '0x6048ac7bb25389994ae13ec9ad40cddfdc33dc0a',
  'query_name': 'airdrop_nusers',
  'value': 1},
 {'id': '0xc03104f0878b4aeb1dcbd5395076e2e8b846b6e113e434322705c61c63be622e',
  'category': 'Airdrop',
  'chain': 'All Superchain',
  'address': '0x6048ac7bb25389994ae13ec9ad40cddfdc33dc0a',
  'query_name': 'airdrop_sumtrxns',
  'value': 0.23208187}]

In [10]:
apps.iloc[84]

id                        0x937cfdb5d95278dc581acb133618064b2b8edfcfd5bc...
recipient                        0xA8F0048A0d1A04663Ca5010d0bEaC5BCAEeA0eef
time                                                             1725772217
name                                                          Attesta_Mesta
status                                                              pending
profile_name                                                    cryptoreumd
profile_url                                https://warpcast.com/cryptoreumd
metadata_name                                                 Attesta_Mesta
metadata_website                https://www.youtube.com/watch?v=895YcSNojjo
project_type                                                            App
category                                                           Payments
address_type                                                       contract
address                          0x42d41fe67832eaead51b75d61a132d16a86892f7
chain_id    

In [6]:
#payments

results_payment = query_project_metrics(apps.iloc[84])
results_payment

2024-10-10 08:16:24,349 - ERROR - Query execution failed: 1 validation error for CreateQueryRunRpcParams
sql
  none is not an allowed value (type=type_error.none.not_allowed)
2024-10-10 08:16:24,350 - ERROR - Query: None
2024-10-10 08:16:24,352 - INFO - Query Attempt: id: 0x937cfdb5d95278dc581acb133618064b2b8edfcfd5bcb95442e86939390abf55, category: Payments, chain: Base, address: 0x42d41fe67832eaead51b75d61a132d16a86892f7, Status: failed, Execution Time: 0.00 seconds, Value: None
2024-10-10 08:16:38,518 - INFO - Query Attempt: id: 0x937cfdb5d95278dc581acb133618064b2b8edfcfd5bcb95442e86939390abf55, category: Payments, chain: Base, address: 0x42d41fe67832eaead51b75d61a132d16a86892f7, Status: success, Execution Time: 14.17 seconds, Value: 0.001
2024-10-10 08:16:45,310 - INFO - Query Attempt: id: 0x937cfdb5d95278dc581acb133618064b2b8edfcfd5bcb95442e86939390abf55, category: Payments, chain: Base, address: 0x42d41fe67832eaead51b75d61a132d16a86892f7, Status: success, Execution Time: 6.79 seco

[{'id': '0x937cfdb5d95278dc581acb133618064b2b8edfcfd5bcb95442e86939390abf55',
  'category': 'Payments',
  'chain': 'Base',
  'address': '0x42d41fe67832eaead51b75d61a132d16a86892f7',
  'query_name': 'payment_nusers',
  'value': None},
 {'id': '0x937cfdb5d95278dc581acb133618064b2b8edfcfd5bcb95442e86939390abf55',
  'category': 'Payments',
  'chain': 'Base',
  'address': '0x42d41fe67832eaead51b75d61a132d16a86892f7',
  'query_name': 'payment_sumtrxns',
  'value': 0.001},
 {'id': '0x937cfdb5d95278dc581acb133618064b2b8edfcfd5bcb95442e86939390abf55',
  'category': 'Payments',
  'chain': 'Base',
  'address': '0x42d41fe67832eaead51b75d61a132d16a86892f7',
  'query_name': 'payments_ntrxns',
  'value': 1}]